Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Works best with 3 convolutional layers.

In [37]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  # Your Code
  def on_epoch_end(self, epochs, logs = {}):
    if logs.get('acc') > 0.999:
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True


callbacks = myCallback()

--2020-09-09 07:24:18--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 74.125.199.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2020-09-09 07:24:18 (190 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [49]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
                                    # Your Code Here
                                    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    # The second convolution
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    # The third convolution
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    # The fourth convolution
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    # The fifth convolution
                                    # tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                    # tf.keras.layers.MaxPooling2D(2,2),
                                    # Flatten the results to feed into a DNN
                                    tf.keras.layers.Flatten(),
                                    # 512 neuron hidden layer
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('happy') and 1 for the other ('sad')
                                    tf.keras.layers.Dense(1, activation='sigmoid')
])

print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 15, 15, 64)       

In [50]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [51]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1/255)

train_generator = train_datagen.flow_from_directory(
    '/tmp/h-or-s',
    target_size = (150,150),
    batch_size = 128,
    class_mode='binary'
)

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [56]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit(
    train_generator,  
      epochs = 20,
      # steps_per_epoch = 8, # error loading data if this is uncommented (batch)
      verbose = True,
      callbacks = [callbacks]
)
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1128 - acc: 0.9500
Epoch 2/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1757 - acc: 0.9125
Epoch 3/20
1/1 [==============================] - ETA: 0s - loss: 0.1026 - acc: 1.0000
Reached 99.9% accuracy so cancelling training!
1/1 [==============================] - 0s 1ms/step - loss: 0.1026 - acc: 1.0000
